In [5]:
import joblib
import requests
import pandas as pd
import numpy as np
try:
    women_model = joblib.load("women_safety_model.pkl")
    features = joblib.load("model_features.pkl")
    print("Model and feature list loaded successfully!") ###(---\____/---+-+-+-*****"""::::::")
except FileNotFoundError as e:
    print(f"Error: {e}")
    print("Ensure 'women_safety_model.pkl' and 'model_features.pkl' are in the same directory.")
    exit()
OPENCAGE_API_KEY = "bba679dd12da4342ac1fb4a72b7959bc"
def get_coordinates_from_address(address):
    try:
        url = f"https://api.opencagedata.com/geocode/v1/json?q={address}&key={OPENCAGE_API_KEY}"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        if data['results']:
            lat = data['results'][0]['geometry']['lat']
            lon = data['results'][0]['geometry']['lng']
            return lat, lon
        else:
            print("No results found for the given address.")
            return None, None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching location data: {e}")
        return None, None
def generate_features(lat, lng, ndf):
    try:
        ndf['distance'] = np.sqrt((ndf['lat'] - lat)**2 + (ndf['long'] - lng)**2)
        nearby_points = ndf.nsmallest(5, 'distance')
        features_dict = {}
        for feature in features:
            if feature not in ['lat', 'long']:
                weights = 1 / (nearby_points['distance'] + 0.001)
                features_dict[feature] = np.average(nearby_points[feature], weights=weights)
        features_dict['lat'] = lat
        features_dict['long'] = lng
        return pd.DataFrame([features_dict], columns=features)
    except Exception as e:
        print(f"Error generating features: {e}")
        return None
def get_safety_recommendation(score):
    if score >= 0.8:
        return "This place is generally safe."
    elif 0.5 <= score < 0.8:
        return "This place is moderately safe. Exercise caution."
    else:
        return "This place is unsafe. Avoid if possible."
def main():
    try:
        ndf = pd.read_csv('data.csv')
    except Exception as e:
        print(f"Error loading dataset: {e}")
        return
    print("Welcome to the Women's Safety Score Predictor!")
    address = input("Enter the location (e.g., Connaught Place, Delhi): ")
    lat, lng = get_coordinates_from_address(address)
    if lat is None or lng is None:
        print("Error: Could not fetch location. Please check the address and try again.")
        return
    print(f"Location found: Latitude = {lat}, Longitude = {lng}")
    input_features = generate_features(lat, lng, ndf)
    if input_features is None:
        print("Could not generate features for prediction.")
        return
    try:
        women_score = women_model.predict(input_features)[0]
    except Exception as e:
        print(f"Error during prediction: {e}")
        return
    final_score = 0
    if 0.4 <= women_score < 0.5:
        final_score = women_score + 0.4
    elif 0.3 <= women_score < 0.4:
        final_score = women_score + 0.37
    elif 0.25 <= women_score < 0.3:
        final_score = women_score + 0.3
    elif 0.5 <= women_score < 0.65:
        final_score = women_score + 0.35
    elif 0.65 <= women_score < 0.75:
        final_score = women_score + 0.18
    elif 0.75 <= women_score < 0.83:
        final_score = women_score + 0.1
    else:
        final_score = women_score
    women_recommendation = get_safety_recommendation(final_score)
    print(f"\nWomen's Safety Score: {final_score:.2f} (Scale: 0-1)")
    print(f"Recommendation: {women_recommendation}")
if __name__ == "__main__":
    main()

Model and feature list loaded successfully!
Welcome to the Women's Safety Score Predictor!
Enter the location (e.g., Connaught Place, Delhi): GB Road, delhi
Location found: Latitude = 28.6504541, Longitude = 77.2206601

Women's Safety Score: 0.20 (Scale: 0-1)
Recommendation: This place is unsafe. Avoid if possible.
